In [1]:
from PIL import Image, ImageFilter, ImageEnhance
import cv2
import imghdr
import pandas as pd
import numpy as np
import os, logging
import pytesseract
import re
from datetime import datetime

VIDEO = '0725.mov'

# Set of allowed image formats
IMAGE_FORMATS = {'png', 'jpg', 'jpeg'}

# Create empty dataframe
df = pd.DataFrame(columns=['IGN', 'Relic Donation Points', 'Date'])

def slice_video_on_profile_change(video_path, output_folder, threshold=100):

    # Open video
    cap = cv2.VideoCapture(video_path)

    # Keep previous frame to compare with
    prev_img = None

    # Counter for sliced segment numbers
    seg_num = 0

    # Read video frames until end
    while cap.isOpened():

        # Read next frame
        ret, frame = cap.read()

        # Break out of loop if video ended
        if not ret:
            break

        # Convert colored frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Compare current frame vs previous
        # If difference bigger than threshold, profile changed
        if prev_img is None or np.mean(np.abs(gray - prev_img)) > threshold:

            # Save current frame as new profile
            prev_img = gray.copy()

            # Increment segment counter
            seg_num += 1

            # Write sliced segment frame to file
            cv2.imwrite(f"{output_folder}/segment_{seg_num}.jpg", frame)

    # Release video resource
    cap.release()


def ocr_image(image_path, kernel_size=1, contrast_factor=2):

    # Validate image file
    ext = os.path.splitext(image_path)[1].lower()
    if ext[1:] not in IMAGE_FORMATS and imghdr.what(image_path) not in IMAGE_FORMATS:
        return None

    # Load image
    image = Image.open(image_path)

    # Convert to grayscale
    gray = image.convert('L')
    # Increase contrast
    enhancer = ImageEnhance.Contrast(gray)
    contrast = enhancer.enhance(contrast_factor)

    # Apply gaussian blur to remove noise
    blur = contrast.filter(ImageFilter.GaussianBlur(radius=kernel_size))

    # Perform OCR
    text = pytesseract.image_to_string(blur)

    return text


slice_video_on_profile_change(VIDEO, 'relicDonations')

folder = 'relicDonations/'

for filename in os.listdir(folder):
    image_path = os.path.join(folder, filename)

    text = ocr_image(image_path)

    # Check if 'Relic' appears in OCR text
    if 'Relic' not in text:
        os.remove(image_path)
        continue

ValueError: invalid literal for int() with base 10: ''

In [8]:
# Loop through images
for filename in sorted(os.listdir(folder)):

    # Full image path
    image_path = os.path.join(folder, filename)
    
    # Check file extension
    ext = os.path.splitext(filename)[1].lower()
    if ext not in ['.jpg', '.jpeg', '.png']:
        continue
    
    print(f"Processing {image_path}")
    # Extract data from image
    text = ocr_image(image_path)
    
    username = re.search(r'([\w]+)(?=\n)', text).group(1)
    
    # Get last number
    points = text.split(" ")[-1]
    # Remove non-digits
    points = int(re.sub(r'[^\d]', '', points))
    date = datetime.fromtimestamp(os.path.getmtime(image_path)).date()

    # Create new row
    new_row = {
        'IGN': username,
        'Relic Donation Points': points,
        'Date': date,
        'File': image_path
    }
    # Append row to dataframe
    df = df.append(new_row, ignore_index=True)

    # Drop duplicates
    df = df.drop_duplicates()

# Export to CSV
df.to_csv('relicDonations.csv', index=False)

Processing relicDonations/segment_41.jpg
Processing relicDonations/segment_54.jpg
Processing relicDonations/segment_81.jpg
Processing relicDonations/segment_56.jpg
Processing relicDonations/segment_43.jpg
Processing relicDonations/segment_47.jpg
Processing relicDonations/segment_51.jpg
Processing relicDonations/segment_45.jpg
Processing relicDonations/segment_22.jpg
Processing relicDonations/segment_9.jpg
Processing relicDonations/segment_37.jpg
Processing relicDonations/segment_20.jpg
Processing relicDonations/segment_30.jpg
Processing relicDonations/segment_33.jpg
Processing relicDonations/segment_26.jpg
Processing relicDonations/segment_28.jpg
Processing relicDonations/segment_3.jpg
Processing relicDonations/segment_7.jpg
Processing relicDonations/segment_12.jpg
Processing relicDonations/segment_5.jpg
Processing relicDonations/segment_60.jpg
Processing relicDonations/segment_77.jpg
Processing relicDonations/segment_67.jpg
Processing relicDonations/segment_71.jpg


In [13]:
# Troubleshooting
text = ocr_image('relicDonations/segment_41.jpg')

#username = re.search(r'([\w]+)(?=\n)', text).group(1)
#print(username)
text

'@ Boss Vanquisher\nSteveth\n\nf Master 2\nWISTERIA\n\nRelic Donation\nPoints\n\nEAperatots\noints\n405 243,600\n'

In [9]:
df

,IGN,Relic Donation Points,Date,File
0,Vanquisher,243600,2023-07-25,relicDonations/segment_41.jpg
1,Boi,33800,2023-07-25,relicDonations/segment_54.jpg
2,xChocobo,19300,2023-07-13,relicDonations/segment_81.jpg
3,Angel,143800,2023-07-25,relicDonations/segment_56.jpg
4,Grandmaster,92500,2023-07-25,relicDonations/segment_43.jpg
5,Cloudbuster,18200,2023-07-25,relicDonations/segment_47.jpg
6,Timekeeper,63500,2023-07-25,relicDonations/segment_51.jpg
7,500,308700,2023-07-25,relicDonations/segment_45.jpg
8,Friend,110800,2023-07-25,relicDonations/segment_22.jpg
9,Overlord,447900,2023-07-25,relicDonations/segment_9.jpg
